# Práctica 3.4
## Procesamiento del lenguaje natural
### Topic modeling vs K-Means

In [11]:
# Corpus de documentos de ejemplo
corpus = ["el cielo es azul", "las nubes son blancas", "el sol brilla",
          "el cielo está despejado", "las estrellas brillan en la noche"]

1) importa las siguientes librerías relacionadas con la vectorización de texto (CountVectorizer y TfidfVectorizer) y la modelo LDA (LatentDirichletAllocation):

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score

2) crea una variable que guarde el resultado de vectorizar (aplicando fit y transform) el corpus con las frases mediante CountVectorizer. Además del vectorizador, guarda en otra variable el diccionario de términos generado.

In [13]:
# Vectorizador de términos
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

3) ahora generaremos el modelo. El número de temas que resultan de una agrupación más ajustada o coherente de términos no es algo que arroja el modelo, sino algo que hay que decidir de antemano. O dicho de otro modo, debemos por prueba y error indicar al
modelo qué número de temas queremos que el modelo genere, haciendo ajustes sucesivos. Comenzaremos pues probando, por ejemplo, con 2 temas:

In [14]:
# Probamos con 2 temas
no_topics = 2

# Ajustamos el modelo LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=10, random_state=0)
lda_model.fit(X)

# Obtenemos la distribución de temas para cada documento
doc_topic_distribution = lda_model.transform(X)

# Obtenemos la distribución de palabras para cada tema
topic_word_distribution = lda_model.components_

4) finalmente, una vez preparado y ejecutado el modelo, necesitamos generar una salida por pantalla que nos permita mostrar los resultados que este arroja.
Por ejemplo, nos puede interesar mostrar los diferentes temas identificados por el modelo (que deberemos “etiquetar” en un segundo momento, es decir, darles un nombre porque el modelo no genera por sí mismo los nombre de los temas) junto a los términos y documentos asociados al mismo. Escribe con tus palabras en el mismo código cuál es el significado de cada término recibido. 
Lleva a tu cuaderno la siguiente función para generar dicha salida:

In [15]:
print("\nDistribución de palabras para cada tópico:")
for i, topic_words in enumerate(topic_word_distribution):
    top_words_indices = topic_words.argsort()[-5:][::-1]
    top_words = [vectorizer.get_feature_names_out()[index] for index in top_words_indices]
    print(f"Tópico {i + 1}: {top_words}")


Distribución de palabras para cada tópico:
Tópico 1: ['el', 'cielo', 'es', 'azul', 'está']
Tópico 2: ['las', 'noche', 'la', 'en', 'estrellas']


5) El código anterior debería arrojar un resultado similar al siguiente:

6) Finalmente, puede ser aconsejable usar algún tipo de métrica que nos ayude a
evaluar la coherencia de los resultados obtenidos. Una opción para modelos tipo LDA
es recurrir a métricas como la puntuación de silueta (silhouette_score).
Modifica el código del ejemplo previo con las siguientes líneas:

In [16]:

# Probamos con 2 temas
no_topics = 2

# Ajustamos el modelo LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=10, random_state=0)
lda_model.fit(X)

doc_topic_distribution = lda_model.transform(X)
topic_word_distribution = lda_model.components_

# Calcular la coherencia de los tópicos
coherence_scores = []
for i, topic_words in enumerate(topic_word_distribution):
    top_words_indices = topic_words.argsort()[-5:][::-1]
    top_words = [vectorizer.get_feature_names_out()[index] for index in top_words_indices]
    coherence_score = silhouette_score(X[:, top_words_indices], labels=doc_topic_distribution.argmax(axis=1))
    coherence_scores.append(coherence_score)

# Mostrar la coherencia de los tópicos
print(f"\nCoherencia de los tópicos para {no_topics} temas:")
for i, coherence_score in enumerate(coherence_scores):
    print(f"Tópico {i + 1}: {coherence_score}")


Coherencia de los tópicos para 2 temas:
Tópico 1: 0.37228015922865254
Tópico 2: 0.5211145618000168


Ejercicios
1) Exploración de temas:
    * Modifica el código anterior encapsulando la creación y salida de resultados del modelo LDA utilizando creando para ello una función llamada “explora_lda” que reciba como argumento el número de temas que LDA debe generar.
    * Observa cómo cambian las distribuciones de temas y palabras con diferente número de temas (es decir, escogiendo 2, 3 y 4 temas, por ejemplo).
    * Pregunta: ¿qué influencia tiene la elección del número de temas en la interpretación y utilidad del modelo?

In [17]:
def create_model(X, topic_number):
    lda_model = LatentDirichletAllocation(n_components=topic_number, max_iter=10, random_state=0)
    lda_model.fit(X)

    doc_topic_distribution = lda_model.transform(X)
    topic_word_distribution = lda_model.components_

    return lda_model, doc_topic_distribution, topic_word_distribution

In [30]:
model, doc_topic_distribution, topic_word_distribution = create_model(X, 3)

print("\nDistribución de palabras para cada tópico:")
for i, topic_words in enumerate(topic_word_distribution):
    top_words_indices = topic_words.argsort()[-5:][::-1]
    top_words = [vectorizer.get_feature_names_out()[index] for index in top_words_indices]
    print(f"Tópico {i + 1}: {top_words}")


Distribución de palabras para cada tópico:
Tópico 1: ['el', 'cielo', 'brilla', 'sol', 'está']
Tópico 2: ['las', 'sol', 'brilla', 'nubes', 'blancas']
Tópico 3: ['brillan', 'noche', 'la', 'estrellas', 'en']


- Cuando se eligen más temas el modelo tiende a ser más disperso y generar topics que se ajusten más a documentos individuales, cuando hay pocos temas, estos temas dan una vision más global del conjunto de documentos

# 2) Implementación de K-Means

1) importamos las librerías necesarias:

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

2) crea un objeto TfidfVectorizer y úsalo para vectorizar (aplicando fit y transform)
el corpus con las frases. Además del vectorizador, guarda en una variable terms el
diccionario de términos generado.

In [27]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
terms_dictionary = vectorizer.get_feature_names_out()
terms_dictionary

array(['azul', 'blancas', 'brilla', 'brillan', 'cielo', 'despejado', 'el',
       'en', 'es', 'estrellas', 'está', 'la', 'las', 'noche', 'nubes',
       'sol', 'son'], dtype=object)

3) generamos el modelo buscando 3 temas (prueba también con otros valores):

In [47]:
# Probamos con 3 agrupaciones
true_k = 3

model = KMeans(n_clusters=true_k, init="k-means++", max_iter=50, n_init=1, random_state=5)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]

for i in range(true_k):
    print(f"Cluster {i}: ")
    for ind in order_centroids[i, :5]:
        print(f'{terms_dictionary[ind]}', end=" ")
    print("\n")

Cluster 0: 
son nubes blancas las el 

Cluster 1: 
el cielo sol brilla es 

Cluster 2: 
noche la estrellas brillan en 


4) muestra en formato Pandas la asociación entre cada frase del corpus y el número de
clúster asignado cuando true_k = 3.

In [48]:
import pandas as pd

In [51]:
results = []
for doc in corpus:
    prediction = model.predict(vectorizer.transform([doc]))
    results.append([
        doc,
        prediction[0]
    ])

df = pd.DataFrame(results, columns=["documents", "cluster"])
df

,documents,cluster
0,el cielo es azul,1
1,las nubes son blancas,0
2,el sol brilla,1
3,el cielo está despejado,1
4,las estrellas brillan en la noche,2
